<a href="https://colab.research.google.com/github/amejiyo/WebSum/blob/sea-add/mT5thai/mt5_thai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Disclaimer : 
This notebook contains the finetuned mT5-base model for Thai sentence summarization. The architecture of the model is based on mT5 model and fine-tuned on text-summarization pairs in Thai. 

From [`huggingface/thanathorn/mt5-cpe-kmutt-thai-sentence-sum`](https://huggingface.co/thanathorn/mt5-cpe-kmutt-thai-sentence-sum)

# Method 1: SimpleTransformer

In [ ]:
!pip install simpletransformers==0.63.4

In [2]:
from simpletransformers.t5 import T5Model, T5Args
from torch import cuda

**Simple Tranformers / [T5 Model](https://simpletransformers.ai/docs/t5-model/)**

In [25]:
# Configuring a T5Model
model_args = T5Args()

model_args.max_length = 100
# The max length of the sequence to be generated. Between 0 and infinity. Default to 20.

model_args.length_penalty = 2.0
# Exponential penalty to the length that is used with beam-based generation. (Default to 2.0)
# It is applied as an exponent to the sequence length, which in turn is used to divide 
# the score of the sequence. Since the score is the log likelihood of the sequence (i.e. negative), 
# length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.

# Load T5Model
model = T5Model("t5", "thanathorn/mt5-cpe-kmutt-thai-sentence-sum", use_cuda=cuda.is_available(),
                args=model_args)

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


In [31]:
sentences = [
    "ถ้าพูดถึงขนมหวานในตำนานที่ชื่นใจที่สุดแล้วละก็ต้องไม่พ้น น้ำแข็งใส แน่เพราะว่าเป็นอะไรที่ชื่นใจสุด",

    "สถาบันวิทยาการหุ่นยนต์ภาคสนาม มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี หรือเรียกว่าฟีโบ้ (FIBO) เป็นหน่วยงานระดับคณะ \
ในสังกัดมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี เริ่มเปิดทำการเรียนการสอนเมื่อ พ.ศ. 2538 ในชื่อ ศูนย์ปฏิบัติการพัฒนาหุ่นยนต์ภาคสนาม \
ตามแผนพัฒนาการศึกษาฉบับที่ 7 เพื่อวิจัยพัฒนาการศึกษาระดับสูงทางด้านวิชาการหุ่นยนต์ และงานวิจัยทางด้านระบบอัตโนมัติในภาคอุตสาหกรรม \
รวมถึงการให้บริการรับปรึกษาด้านอุตสาหกรรมต่างๆภายในประเทศ",

    "หุ่นยนต์ประชาสัมพันธ์นะโม เป็นหุ่นยนต์กึ่งหุ่นยนต์ฮิวแมนนอยด์ ท่อนบนมีลักษณะเป็นหุ่นยนต์ที่มีลักษณะคล้ายมนุษย์ \
ส่วนท่อนล่างเป็นล้อเพื่อเพิ่มความคล่องตัวและเสถียรภาพในการเคลื่อนที่ ประดิษฐ์โดยนักวิจัยของสถาบันวิทยาการหุ่นยนต์ภาคสนาม \
นายบุญเลิศ มณีฉาย, ดร.ถวิดามณีวรรณ์ และ ศูนย์การศึกษาในเมือง(CODE) สนับสนุนทุนโดยมหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี \
เผยแพร่ผลงานเมื่อ 29 มีนาคม พ.ศ. 2553",
]
prediction = model.predict(sentences)

# for p in prediction: print(p)

print("\n<input length> => <output length> | prediction result\n")
for i in range(len(prediction)): 
    print(f"<{len(sentences[i])}> => <{len(prediction[i])}> | {prediction[i]}")

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/3 [00:00<?, ?it/s]


<input length> => <output length> | prediction result

<98> => <33> | น้ําแข็งใสเป็นอะไรที่ชื่นใจที่สุด
<431> => <152> | สถาบันวิทยาการหุ่นยนต์ภาคสนาม มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี เริ่มเปิดทําการเรียนการสอนเมื่อ พ.ศ. 2536 ในชื่อ ศูนย์ปฏิบัติการพัฒนาหุ่นยนต์ภาคสนาม
<388> => <110> | หุ่นยนต์ประชาสัมพันธ์นะโม เป็นหุ่นยนต์กึ่งหุ่นยนต์ฮิวแมนนอยด์ ท่อนบนมีลักษณะเป็นหุ่นยนต์ที่มีลักษณะคล้ายมนุษย์


(Note from author) Here is a short example:

``` python
model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]
```

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

# Method 2: In progress

In [35]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MT5Model

**Hugging face Tranformers / [mT5](https://huggingface.co/docs/transformers/main/en/model_doc/mt5#transformers.MT5ForConditionalGeneration)**

In [36]:
tokenizer = AutoTokenizer.from_pretrained("thanathorn/mt5-cpe-kmutt-thai-sentence-sum")

model = AutoModelForSeq2SeqLM.from_pretrained("thanathorn/mt5-cpe-kmutt-thai-sentence-sum")

# model = MT5Model.from_pretrained("thanathorn/mt5-cpe-kmutt-thai-sentence-sum")
# ??

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of the model checkpoint at thanathorn/mt5-cpe-kmutt-thai-sentence-sum were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (ini